In [1]:
from sklearn import tree, model_selection, metrics, datasets
from pandas import DataFrame
from sklearn.model_selection import GridSearchCV
from scipy.stats.mstats import mquantiles
import numpy as np

Загрузим и изучим датасет boston

In [2]:
boston = datasets.load_boston()
print(boston.DESCR)
feature = boston['data']
target = boston['target']
feature_names = boston['feature_names']

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [3]:
frame = DataFrame(feature)
frame.columns = feature_names
frame['TARGET'] = target
frame.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


Реализуем решающее дерево

In [4]:
class decision_tree:     
    def __init__(self, max_depth, parent_level = 0):
        self.answer = 0
        self.condition = lambda x: True
        self.max_depth = max_depth
        self.level = parent_level + 1
        self.true_child = None
        self.false_child = None
        
    def __feature_target_division__(self, feature, target, predicate):
        return ([x for x in feature if (predicate(x)==True)],
                [target[i] for i in xrange(len(feature)) if (predicate(feature[i])==True)],
                [x for x in feature if (predicate(x)==False)],
                [target[i] for i in xrange(len(feature)) if (predicate(feature[i])==False)])
            
    def __score__(self, feature, target, feature_no, threshold):
        total_len = float(len(feature))
        (feature_less, target_less, feature_more,
         target_more) = self.__feature_target_division__(
            feature, target,lambda x: x[feature_no] <= threshold
        )
        
        less_len = float(len(target_less))
        more_len = float(len(target_more))
        
        return (less_len*np.var(target_less) + more_len*np.var(target_more))/total_len
    
    def __matrix_argmin__(self, a):
#        print(a)
        x, y = np.shape(a)
        r = np.argmin(a)
#        print(r//y, r % y)
        return r // y, r % y
        
    def __find_best_subdivision__(self, feature, target):
        feature_values = np.array(feature).T
        threshold_values = [np.linspace(np.min(value_list), np.max(value_list),10)[1:-1]
                                       for value_list in feature_values] 
        scores = [[self.__score__(feature, target, feature_no, threshold)
                   for threshold in threshold_values[feature_no]]
                  for feature_no in xrange(len(feature_values))] 
        
        feature_no, threshold_index = self.__matrix_argmin__(scores)
        
        return feature_no, threshold_values[feature_no][threshold_index]
    
    def fit(self, feature, target):
#        print(np.shape(feature), np.shape(target))    
        
        if(len(target) > 0):
            self.answer = np.mean(target)
        else:
            self.answer = None
        
        if(self.level <= self.max_depth and len(target) > 0 and len(feature) > 0):
            if type(feature)==np.ndarray:
                feature = feature.tolist()
            
            feature_no, threshold = self.__find_best_subdivision__(feature, target)
            self.condition = lambda x: x[feature_no] <= threshold
            
            (true_child_feature, true_child_target, false_child_feature,
             false_child_target) = self.__feature_target_division__(feature, target,
                                                                    self.condition)
            
            for item in feature:
                del item[feature_no]
            
            if(len(true_child_target) > 0):            
                self.true_child = decision_tree(self.max_depth, parent_level=self.level)
                self.true_child.fit(true_child_feature, true_child_target)
                
            if(len(false_child_target) > 0):
                self.false_child = decision_tree(self.max_depth, parent_level=self.level)
                self.false_child.fit(false_child_feature, false_child_target)
            
    
    def __answer__(self, item):
        if self.condition(item) == True and self.true_child is not None:
            return self.true_child.__answer__(item)
        elif self.condition(item) == False and self.false_child is not None:
            return self.false_child.__answer__(item)
        else:
            return self.answer
    
    def predict(self, feature):
        return [self.__answer__(x) for x in feature]

Применим решающее дерево к выборке, разделив её на обучающую и тестовую подвыборки. Посмотрим на результаты. MSE должно быть значительно меньше стандартного отклонения в target тестовой подвыборки (иначе можно было бы получить настолько же качественный ответ, выдавая его случайно)

In [5]:
feature_train, feature_test, target_train, target_test = model_selection.train_test_split(
    feature, target, test_size=0.25)

In [6]:
max_depth = 2
model = decision_tree(max_depth = max_depth)
model.fit(feature_train, target_train)

In [7]:
result = model.predict(feature_test)
print('MSE: {}'.format(metrics.mean_squared_error(target_test, result)))
print('Стандартное отклонение: {}'.format(np.var(target_test)))

MSE: 28.8348839635
Стандартное отклонение: 73.8739314279


Сравним с реализацией дерева из sklearn

In [8]:
another_model = tree.DecisionTreeRegressor(max_depth=max_depth)
another_model.fit(feature_train, target_train)
print(metrics.mean_squared_error(target_test, another_model.predict(feature_test)))

27.7938550592


Построенное нами дерево немного уступает реализации из sklearn, но в целом выдаёт сопоставимое качество. Не знаю как вы, а я считаю, что это успех.